In [1]:
import numpy as np
import pandas as pd

# Part 1 問題

## Q1. 載入資料＆簡易分析
要求：
1. 請將上述 metadata 資料載入成為 Pandas 的 DataFrame

請回答：
1. 在此份檔案當中，總共有多少筆資料？多少欄位？
2. 在此份資料當中，擁有最多品項的前三名品牌為？

In [2]:
# Demands
P1_df = pd.read_json('./datasets/meta_All_Beauty.json', lines=True)

In [3]:
# P1_df.info()
# P1_df.head()


# Ans
## 1
print(f'rows: {P1_df.shape[0]}, columns:{P1_df.shape[1]}')

## 2
brands_cleaned = P1_df.brand.apply(str).replace('', np.nan)
print('最多品項的前三名品牌為:\n',brands_cleaned.dropna().value_counts().head(3))


rows: 32892, columns:19
最多品項的前三名品牌為:
 VAGA               252
L'Oreal Paris      113
Philips Norelco     88
Name: brand, dtype: int64


## Q2. 清理資料＆調整格式
要求：

1. 移除資料中不必要的欄位，只留下 `'asin', 'brand', 'title', 'price', 'rank', 'description'`
2. 將銷售排名 `rank` 中的排名字串抽取出來、轉為數字

請回答：

1. 找到資料中銷售排名最高的產品是第幾名？
2. 最高銷售排名的產品為何？

In [4]:
# Demands

## 1
P1_df_cleaned = P1_df[['asin', 'brand', 'title', 'price', 'rank', 'description']].copy()

## 2
#type(P1_df_cleaned['rank'][0])
rank_split = P1_df_cleaned['rank'].str.split(' ', n=1, expand=True)
rank_numbers = pd.to_numeric(rank_split[0].str.split(',').str.join(''))
P1_df_cleaned['rank'] = rank_numbers

In [5]:
# Ans

## 1
print(f'資料中銷售排名最高的產品是第 {np.min(rank_numbers)} 名')

## 2
no1 = P1_df_cleaned.sort_values(by=['rank'])['title'].values[0]
print(f'最高銷售排名的產品為:\n {no1}')

資料中銷售排名最高的產品是第 35.0 名
最高銷售排名的產品為:
 Braun Clean &amp; Renew Refill Cartridges CCR, 4 Count (Packaging May Vary)


# Part 2 問題

## Q3. 聚合資料

要求：

1. pandas 載入美妝品的評價資料

請回答：

1. 找到評價數最多的產品其 asin
2. 上述產品的評價次數為？
3. 上述產品的平均評價為？

In [6]:
# Demands
P2_df = pd.read_json('./datasets/All_Beauty.json', lines=True)
# P2_df.info()

In [7]:
# P2_df.asin.value_counts()

# Ans
## Ans1
most_reivewed_asin = P2_df.asin.value_counts().index[0]
print(f'價數最多的產品為:\n {most_reivewed_asin}')

## Ans2
most_reivewed_times = P2_df.asin.value_counts().values[0]
print(f'上述產品的評價次數為:\n {most_reivewed_times}')

## Ans3
most_reivewed_overall_mean = P2_df.groupby('asin')['overall'].mean()[most_reivewed_asin]
print(f'上述產品的平均評價為:\n {most_reivewed_overall_mean}')

價數最多的產品為:
 B000FOI48G
上述產品的評價次數為:
 8672
上述產品的平均評價為:
 4.393450184501845


## Q4. 結合兩張表格
要求：

1. 將 Q2 取得的產品資料 DataFrame，加入 Q4 的評價資料作為欄位
    1. `overall_mean` 紀錄產品的平均評分
    2. `reviews_count` 紀錄評價次數

請回答：

1. 最高銷售排名的產品其使用者平均的評分？
2. 最高銷售排名的產品其使用者評價次數？

In [8]:
# Demands
overall_means = P2_df.groupby('asin')['overall'].mean()
reivewed_times = P2_df.groupby('asin')['overall'].count()
reviewd_data = pd.merge(left=overall_means, right=reivewed_times, left_index=True, right_index=True)
reviewd_data.columns = ['overall_mean', 'reviews_count']
df_merged = pd.merge(left=P1_df_cleaned, right=reviewd_data, left_on='asin', right_index=True)
df_merged.head(2)

,asin,brand,title,price,rank,description,overall_mean,reviews_count
0,6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,2938573.0,[Loud 'N Clear Personal Sound Amplifier allows...,2.5,2
1,7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,872854.0,[No7 Lift & Luminate Triple Action Serum 50ml ...,3.0,1


In [9]:
# Ans
no1_full = df_merged[df_merged['title']==no1]

## 1
print(f'最高銷售排名的產品其使用者平均的評分為:\n {no1_full.overall_mean.values[0]}')
## 2
print(f'最高銷售排名的產品其使用者評價次數為:\n {no1_full.reviews_count.values[0]}')

最高銷售排名的產品其使用者平均的評分為:
 4.846153846153846
最高銷售排名的產品其使用者評價次數為:
 13
